# Slot-to-Coefficient matrix

We define the matrix $U_n$ as
$$
U_n = \begin{pmatrix}
    1 & \zeta_0 & \zeta_0^2 & \cdots & \zeta_0^{n-1} \\
    1 & \zeta_1 & \zeta_1^2 & \cdots & \zeta_1^{n-1} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    1 & \zeta_{n-1} & \zeta_{n-1}^2 & \cdots & \zeta_{n-1}^{n-1}
\end{pmatrix}
$$
where $\zeta_i = e^{2\pi I \cdot 5^i / {2N}}$ are $2N$-th roots of unity.
The full $N/2 \times N/2$ matrix needed for the Slot-to-Coefficient transform is then given by the following rule:
Its non-zero diagonals are indexed by $i=0,\dots,n-1$
 and its $i$-th diagonal is a concatenation of $N/(2n)$ copies of the $i$-th diagonal of $U_n$.


In [10]:
load("dft_utils.sage")

In [11]:
def construct_dft_matrix(dim, N):
    # This outputs the quadratic matrix of dimension dim x dim in a diagonal dictionary
    # It's the Vandermonde matrix of the slot-to-coefficient transform of dimensions dim x dim
    # N is the ring degree
    return {i: [root(N, pow(5, j, 2*N) * (i + j) % (2*N)) for j in range(dim)] for i in range(dim)}

def extend_matrix(M, new_dim):
    # This extends a square matrix M of dimension dim x dim,
    # to a block matrix with dimensions new_dim x new_dim,
    # whose non-zero diagonals are indexed by i =0,…,n-1, 
    # and whose i-th diagonal is a concatenation of new_dim/dim
    # copies of the i-th diagonal of M
    # The matrix is provided by a diagonal dictionary,
    # and its indices must be given by the integers 0, 1, ..., dim-1.
    dim = len(M[0])
    assert new_dim % dim == 0
    if new_dim == dim: return M
    B = new_dim // dim # number of blocks
        
    for j in M:
        M[j] = M[j] * B
    return M
    
def diagonal_sanity_check(M):
    for i in M:
        assert len(M[i]) == len(M[0]), f"Matrix is not square {i}"
        assert 0 <= i < len(M[0]), f"Index {i} is out of bounds"

def to_matrix(M):
    # Represents a matrix from its diagonal dictionary
    dim = len(M[0])
    A = Matrix(ComplexField(15), dim, dim, sparse=True)
    for j, diag in M.items():
        for i, val in enumerate(diag):
            A[i, i + j - dim] = val
    return A

In [12]:
# Test
if __name__ == "__main__":
    d = construct_dft_matrix(4, 4)
    print(to_matrix(d))
    print()
    e = extend_matrix(d, 8)
    diagonal_sanity_check(e)
    print(to_matrix(e))

[               1.000  6.123e-17 + 1.000*I -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I]
[               1.000  6.123e-17 + 1.000*I -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I]
[               1.000  6.123e-17 + 1.000*I -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I]
[               1.000  6.123e-17 + 1.000*I -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I]

[               1.000  6.123e-17 + 1.000*I -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I               0.0000               0.0000               0.0000               0.0000]
[              0.0000  6.123e-17 + 1.000*I -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I                1.000               0.0000               0.0000               0.0000]
[              0.0000               0.0000 -1.000 + 1.225e-16*I -1.837e-16 - 1.000*I                1.000  6.123e-17 + 1.000*I               0.0000               0.0000]
[              0.0000               0.0000               0.0000 -1.837e-16 - 1.000*I                1.000  6.123e-17 + 1.000*I -1.000 + 1.225e-16